# Particle Flow Test 1

Make our own particle flow network, train it on photon jets

In [1]:
# Up to a couple minutes to import everything

# Computing imports
print(f"Importing computing things...")
import os
import numpy as np

print(f"Importing tensorflow... (this could take a while)")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # Make tensorflow quieter
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay

# Useful imports
print(f"Importing display things...")
from tqdm import tqdm
import matplotlib.pyplot as plt

# Energyflow imports
print(f"Importing energyflow things...")
import energyflow as ef
from energyflow.utils import data_split, to_categorical

Importing computing things...
Importing tensorflow... (this could take a while)
Importing display things...
Importing energyflow things...


## Import, preprocess data

Data lives in `X_train`, `X_val`, `X_test`, and `Y_train`, `Y_val`, `Y_test`.

In [2]:
# ~5 sec
jets_path = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data/processed/scalar_test/all_jets_point_cloud.npz"
jets = np.load(jets_path)

X, y = jets["X"], jets["y"]
Y = to_categorical(y)

print("Loaded data")

Loaded data


In [3]:
# ~5 sec
# Do train/val/test split
n_val = 20000
n_test = 20000

(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=n_val, test=n_test)

print("Done train/val/test split")

Done train/val/test split


## Define the model

PFN model:
$$\text{PFN}=F\left(\sum_{i=1}^M \Phi(p_i)\right)$$

In [4]:
def PFN(n_features,
        n_particles,
        n_outputs,
        Phi_sizes,
        F_sizes,
        name=None):
    
        inputs = layers.Input((n_particles, n_features), name="input")    
        masking_layer = layers.Masking(
            mask_value=0.,
            input_shape=(n_particles, n_features)
        )
        Phi_layers = [layers.Dense(size, activation="relu", name=f"Phi_{i}") for i, size in enumerate(Phi_sizes)]
        F_layers = [layers.Dense(size, activation="relu", name=f"F_{i}") for i, size in enumerate(F_sizes)]
        last_layer = layers.Dense(n_outputs, name="output")
        
        x = masking_layer(inputs)
        for layer in Phi_layers:
            x = layers.TimeDistributed(layer)(x)
        x = tf.math.reduce_sum(x, axis=1)
        for layer in F_layers:
            x = layer(x)
        x = last_layer(x)
        
        return tf.keras.Model(inputs=inputs, outputs=x)

## Compile and train the model

In [5]:
tf.keras.backend.clear_session()
print(f"GPU memory usage: {tf.config.experimental.get_memory_info('GPU:0')}")

GPU memory usage: {'current': 0, 'peak': 0}


In [6]:
# Takes about 5 sec the first time
# Data dimensions
n_features = 4
n_particles = 960

model = PFN(
    n_features=n_features,
    n_particles=n_particles,
    n_outputs=3,
    Phi_sizes=[100, 100, 256],
    F_sizes=[100, 100, 100]
)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 960, 4)]          0         
                                                                 
 masking (Masking)           (None, 960, 4)            0         
                                                                 
 time_distributed (TimeDistr  (None, 960, 100)         500       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 960, 100)         10100     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 960, 256)         25856     
 tributed)                                                       
                                                             

In [8]:
lr = 2e-4

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [9]:
history = model.fit(X_train, Y_train,
                    epochs=15,
                    batch_size=100,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/15
2600/2600 [==============================] - 58s 15ms/step - loss: 45.7914 - accuracy: 0.3330 - val_loss: 10.8852 - val_accuracy: 0.3298
Epoch 2/15
2600/2600 [==============================] - 37s 14ms/step - loss: 6.5164 - accuracy: 0.3339 - val_loss: 5.9408 - val_accuracy: 0.3340
Epoch 3/15
2600/2600 [==============================] - 37s 14ms/step - loss: 4.7328 - accuracy: 0.3328 - val_loss: 2.1628 - val_accuracy: 0.3298
Epoch 4/15
2600/2600 [==============================] - 37s 14ms/step - loss: 2.6926 - accuracy: 0.3325 - val_loss: 1.1942 - val_accuracy: 0.3340
Epoch 5/15
2600/2600 [==============================] - 37s 14ms/step - loss: 1.5013 - accuracy: 0.3325 - val_loss: 1.1006 - val_accuracy: 0.3340
Epoch 6/15
2600/2600 [==============================] - 37s 14ms/step - loss: 1.8768 - accuracy: 0.3327 - val_loss: 1.1896 - val_accuracy: 0.3390
Epoch 7/15
2600/2600 [==============================] - 37s 14ms/step - loss: 1.1141 - accuracy: 0.3345 - val_loss: 1.0986

In [ ]:
# ~5 sec
model.save("/usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/models/pfn/big_deep_deep_pfn_4.model")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# get predictions on test data
preds = model.predict(X_test, batch_size=500)

In [ ]:
test_labels = np.argmax(Y_test, axis=1)
pred_labels = np.argmax(preds, axis=1)

mask = (test_labels == pred_labels).astype(float)
print(mask)
print(f"Test accuracy: {mask.mean()}")

In [ ]:
cm = confusion_matrix(test_labels, pred_labels).astype(float)
cm /= np.sum(cm, axis=0, keepdims=True)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["pion", "photon", "scalar"]
);
disp.plot(cmap="Blues");

print(f"Total test jets: {len(test_labels)}")